'2011TO2011.7z'

In [ ]:
for mes in meses:
            download_link = f"ftp://ftp.mtps.gov.br/pdet/microdados/CAGED/{ano}/CAGEDEST_{mes}{ano}.7z"
            download_caged_file(download_link, ano, mes, raw_path)

In [1]:
## caged_brach
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

pd.options.display.max_columns = 999
pd.options.display.max_rows = 1999
pd.options.display.max_colwidth = 200

from paths import *
from scpts import manipulation
from scpts import io
from scpts import caged

from basedosdados import Storage, Table, Dataset
import basedosdados as bd

import glob


### Download Dados
 - ftp://ftp.mtps.gov.br/pdet/microdados/CAGED/
 - ftp://ftp.mtps.gov.br/pdet/microdados/CAGED_AJUSTES/
 - ftp://ftp.mtps.gov.br/pdet/microdados/NOVO%20CAGED/

In [2]:
### CAGED ANTIGO
caged.caged_antigo_download()

### CAGED AJUSTES 
caged.caged_antigo_ajustes_download()

### CAGED AJUSTES
caged.caged_antigo_ajustes_2002a2009_download()

#===== CAGED ANTIGO =====#

01/2007 | já existe
02/2007 | já existe
03/2007 | já existe
04/2007 | já existe
05/2007 | já existe
06/2007 | já existe
07/2007 | já existe
08/2007 | já existe
09/2007 | já existe
10/2007 | já existe
11/2007 | já existe
12/2007 | já existe
01/2008 | já existe
02/2008 | já existe
03/2008 | já existe
04/2008 | já existe
05/2008 | já existe
06/2008 | já existe
07/2008 | já existe
08/2008 | já existe
09/2008 | já existe
10/2008 | já existe
11/2008 | já existe
12/2008 | já existe
01/2009 | já existe
02/2009 | já existe
03/2009 | já existe
04/2009 | já existe
05/2009 | já existe
06/2009 | já existe
07/2009 | já existe
08/2009 | já existe
09/2009 | já existe
10/2009 | já existe
11/2009 | já existe
12/2009 | já existe
01/2010 | já existe
02/2010 | já existe
03/2010 | já existe
04/2010 | já existe
05/2010 | já existe
06/2010 | já existe
07/2010 | já existe
08/2010 | já existe
09/2010 | já existe
10/2010 | já existe
11/2010 | já existe
12/2010 | já existe
01/2011 | já

In [26]:
## os arquivos de 2002 a 2009 não estão separadas por mês, 
## essa etapa extrai os dados e organiza na forma ano/mes
folders = glob.glob("../data/caged/raw/caged_antigo_ajustes/*/")
caged.caged_antigo_ajustes_2002a2009_extract_organize(folders)

2007 | extraido
2008 | extraido
2009 | extraido


### Padronizacao

- o arquivo CAGED_062012 teve troca de nome quando foi descompactado
&nbsp;
- Particao | ano=XXXX/mex=XX/sigla_uf=XX

In [33]:

##--------------Download basedosdados---------------###
q_municipios = """
SELECT id_municipio,id_municipio_6,sigla_uf
FROM `basedosdados.br_bd_diretorios_brasil.municipio`
"""
# municipios = bd.read_sql(q_municipios, billing_project_id='gabinete-sv')
# municipios['id_municipio'] = municipios['id_municipio'].astype(str)
# municipios['id_municipio_6'] = municipios['id_municipio_6'].astype(str)
# municipios.to_parquet(f'../data/caged/raw/municipios.parquet')


In [56]:
### Padroniza CAGED ANTIGO AJUSTES  
clean_save_path = "../data/caged/clean/caged_antigo_ajustes/"
folders = glob.glob("../data/caged/raw/caged_antigo_ajustes/*/*/")
municipios = pd.read_parquet('../data/caged/raw/municipios.parquet')

caged.caged_antigo_padronize_and_partitioned(
    folders, 
    clean_save_path, 
    municipios, 
    force_remove_csv=False
)

In [53]:
folders = glob.glob("../data/caged/raw/caged_antigo/*/*/")
filtro_anos = [2019]
final_folders = [folder for folder in folders if folder.split('/')[-3] in [str(ano) for ano in filtro_anos]] if filtro_anos else folders

In [55]:
### Padroniza CAGED ANTIGO
clean_save_path = "../data/caged/clean/caged_antigo/"
folders = glob.glob("../data/caged/raw/caged_antigo/*/*/")
municipios = pd.read_parquet('../data/caged/raw/municipios.parquet')



df = caged.caged_antigo_padronize_and_partitioned(
    folders[68:], 
    clean_save_path, 
    municipios, 
    force_remove_csv=False
)

In [6]:
for ano in range(2007,2019):
    os.mkdir('../data/caged/clean/caged_antigo/'  + str(ano))

### SOBE DADOS PARA BASEDOSDADOS | CAGED ANTIGO

In [3]:
st = bd.Storage('microdados_antigos_ajustes','br_me_caged')

In [7]:
# CAGED ANTIGO AJUSTES

tb = Table('microdados_antigos_ajustes','br_me_caged')

# tb.create(
#     path='../data/caged/clean/caged_antigo_ajustes/2007/',
#     if_table_exists='replace',
#     if_storage_data_exists='pass',
#     if_table_config_exists='pass'
# )


# for ano in range(2008,2019):
#     tb.append(
#         filepath=f'../data/caged/clean/caged_antigo_ajustes/{ano}/',
#         partitions='ano=value/mes=value/sigla_uf=value',
#         if_exists='replace'
#     )
#     print(f'{ano} done!\n')
    
tb.publish(if_exists='replace')

In [58]:
# CAGED ANTIGO PADRAO

tb = Table('microdados_antigos','br_me_caged')

# tb.create(
#     path='../data/caged/clean/caged_antigo/2019/',
#     if_table_exists='replace',
#     if_storage_data_exists='replace',
#     if_table_config_exists='pass'
# )


# for ano in range(2012,2019):
#     tb.append(
#         filepath=f'../data/caged/clean/caged_antigo/{ano}/',
#         partitions='ano=value/mes=value/sigla_uf=value',
#         if_exists='replace'
#     )
#     print(f'{ano} done!\n')
    
# tb.publish(if_exists='replace')

### SOBE DADOS PARA BASEDOSDADOS | CAGED NOVO

### DOWNLOAD RAIS


In [2]:
import os
import urllib.request as request
from contextlib import closing
import shutil
import time

In [3]:
def download_data(download_link, download_path, filename):
    ## download do arquivo
    with closing(request.urlopen(download_link)) as r:
        with open(os.path.join(download_path, filename), "wb") as f:
            shutil.copyfileobj(r, f)


In [4]:
for ano in range(2011,2018):
    download_url = f"ftp://ftp.mtps.gov.br/pdet/microdados/RAIS/{ano}"
    
    download_path_year = f'../data/rais/{ano}'
    if not os.path.exists(download_path_year):
            os.mkdir(download_path_year)
            
    with request.urlopen(download_url) as r:
        data = r.read()
    file_names = [line.split(' ')[-1] for line in data.decode().splitlines()]
    
    for file_name in file_names:
        ## cria pastas
        download_link = download_url + f'/{file_name}'
#         print(download_link)
        if os.path.exists(os.path.join(download_path_year, file_name)):
            print(f"{ano}/{file_name} | já existe")
        else:
#             try:
            ti = time.time()
            download_data(download_link, download_path_year, file_name)
            t = time.strftime("%M:%S", time.gmtime((time.time() - ti)))
            print(f"{ano}/{file_name} | criado em {t}")
#             except:
#                 print(f"{ano}/{file_name} | não conseguiu baixar")
    print('\n')

2011/AC2011.7z | já existe
2011/AL2011.7z | já existe
zzz
2011/AM2011.7z | criado em 00:14
zzz
2011/AP2011.7z | criado em 00:02
zzz
2011/BA2011.7z | criado em 00:30
zzz
2011/CE2011.7z | criado em 00:07
zzz
2011/DF2011.7z | criado em 00:04
zzz
2011/ES2011.7z | criado em 00:04
zzz
2011/ESTB2011.7z | criado em 00:13
zzz
2011/GO2011.7z | criado em 00:06
zzz
2011/MA2011.7z | criado em 00:02
zzz
2011/MG2011.7z | criado em 00:47
zzz
2011/MS2011.7z | criado em 00:02
zzz
2011/MT2011.7z | criado em 00:03
zzz
2011/PA2011.7z | criado em 00:08
zzz
2011/PB2011.7z | criado em 00:02
zzz
2011/PE2011.7z | criado em 00:05
zzz
2011/PI2011.7z | criado em 00:01
zzz
2011/PR2011.7z | criado em 00:21
zzz
2011/RJ2011.7z | criado em 00:14
zzz
2011/RN2011.7z | criado em 00:02
zzz
2011/RO2011.7z | criado em 00:02
zzz
2011/RR2011.7z | criado em 00:01
zzz
2011/RS2011.7z | criado em 00:11
zzz
2011/SC2011.7z | criado em 00:08
zzz
2011/SE2011.7z | criado em 00:02
zzz
2011/SP2011.7z | criado em 00:48
zzz
2011/TO2011.7z 